In [2]:
import keras.applications.vgg16
import keras.applications.resnet_v2
import keras.applications.mobilenet_v2
import keras.preprocessing.image
import keras.backend
import math
import numpy
import os
import pathlib

In [16]:
cfg={
    "list_of_labels": list(["manekia", "ottonia", "peperomia", "piper", "pothomorphe"]),
    "list_of_models": list(["mobilenetv2"]),
    "orientation": list(["vertical"]),
    "patches": list([3]),
    "height": 256,
    "width": 256,
}

In [17]:
def get_model(model, **kwargs):
    if model == "vgg16":
        return keras.applications.vgg16.VGG16(**kwargs), keras.applications.vgg16.preprocess_input
    if model == "resnet50v2":
        return keras.applications.resnet_v2.ResNet50V2(**kwargs), keras.applications.resnet_v2.preprocess_input
    if model == "mobilenetv2":
        return keras.applications.mobilenet_v2.MobileNetV2(**kwargs), keras.applications.mobilenet_v2.preprocess_input

    raise ValueError


def get_label(filename, labels):
    return next(labels.index(l) for l in labels if l in filename)


def save(features, labels, model_name, n_patches, orientation):
    path_out = os.path.join("out", model_name)
    pathlib.Path(path_out).mkdir(parents=True, exist_ok=True)
    output_filename = os.path.join(path_out, f"{orientation}+patch={n_patches}+{model_name}")
    numpy.savez_compressed(output_filename, features=features, labels=labels)
    print(output_filename, features.shape, labels.shape)


def get_input_shape(cfg, n_patches, orientation):
    if orientation == "vertical":
        return (cfg["height"], math.floor(cfg["width"] / n_patches), 3)
    if orientation == "horizontal":
        return (math.floor(cfg["height"] / n_patches), cfg["width"], 3)
    if orientation == "h+v":
        return (math.floor(cfg["height"] / n_patches), math.floor(cfg["width"] / n_patches), 3)
    return (cfg["height"], cfg["width"], 3)


def next_patch(orientation, spec, n):
    if orientation == "vertical":
        step = math.floor(spec.shape[1] / n)
        for i in range(n):
            yield spec[:, i * step:(i + 1) * step, :]
    elif orientation == "horizontal":
        step = math.floor(spec.shape[0] / n)
        for i in range(n):
            yield spec[i * step:(i + 1) * step, :, :]

In [18]:
for orientation in cfg["orientation"]:
    if not orientation in ("horizontal", "vertical", "h+v"):
        raise ValueError("orientation not found")
    for model_name in cfg["list_of_models"]:
        for n_patches in cfg["patches"]:
            if n_patches < 1:
                raise ValueError("n_patches should be greater than one")

            keras.backend.clear_session()

            input_shape = get_input_shape(cfg, n_patches, orientation)

            model, preprocess_input = get_model(model_name, weights="imagenet", include_top=False,
                                                input_shape=input_shape, pooling="avg")

            features = []
            labels = []
            for filename in sorted(pathlib.Path("dataset").rglob("*")):
                l = get_label(str(filename.stem), cfg["list_of_labels"])
                print(filename.resolve(), cfg["list_of_labels"][l], model_name, n_patches)
                img = keras.preprocessing.image.load_img(str(filename.resolve()))
                spec = keras.preprocessing.image.img_to_array(img)
                if orientation == "h+v":
                    for p in next_patch("horizontal", spec, n_patches):
                        for q in next_patch("vertical", p, n_patches):
                            q = preprocess_input(q)
                            q = numpy.expand_dims(q, axis=0)
                            features.append(model.predict(q))
                            labels.append(l + 1)
                else:
                    for p in next_patch(orientation, spec, n_patches):
                        p = preprocess_input(p)
                        p = numpy.expand_dims(p, axis=0)
                        features.append(model.predict(p))
                        labels.append(l + 1)

            features = numpy.concatenate(features)
            labels = numpy.array(labels)
            save(features, labels, model_name, n_patches, orientation)

C:\Users\xandao\Documents\GitKraken\deep_feature\dataset\manekia01.png manekia mobilenetv2 3
C:\Users\xandao\Documents\GitKraken\deep_feature\dataset\manekia02.png manekia mobilenetv2 3
C:\Users\xandao\Documents\GitKraken\deep_feature\dataset\manekia03.png manekia mobilenetv2 3
C:\Users\xandao\Documents\GitKraken\deep_feature\dataset\manekia04.png manekia mobilenetv2 3
C:\Users\xandao\Documents\GitKraken\deep_feature\dataset\manekia05.png manekia mobilenetv2 3
C:\Users\xandao\Documents\GitKraken\deep_feature\dataset\manekia06.png manekia mobilenetv2 3
C:\Users\xandao\Documents\GitKraken\deep_feature\dataset\manekia07.png manekia mobilenetv2 3
C:\Users\xandao\Documents\GitKraken\deep_feature\dataset\manekia08.png manekia mobilenetv2 3
C:\Users\xandao\Documents\GitKraken\deep_feature\dataset\manekia09.png manekia mobilenetv2 3
C:\Users\xandao\Documents\GitKraken\deep_feature\dataset\manekia10.png manekia mobilenetv2 3
C:\Users\xandao\Documents\GitKraken\deep_feature\dataset\manekia11.png

In [3]:
x = numpy.load("out/mobilenetv2/vertical+patch=3+mobilenetv2.npz")
numpy.unique(x["labels"])

array([1, 2, 3, 4, 5])

In [4]:
x["features"].shape

(1125, 1280)